# Cython

Dans cette section, nous allons voir comment **améliorer les performances** d'un code Python et comment **interfacer du code C/C++ à Python** en utilisant [**Cython**](http://cython.org/).

Cette section est inspirée du [tutorial](https://www.youtube.com/watch?v=gMvkiQ-gOW8) donné par Kurt Smith à la conférence SciPy de 2015 et de la documentation Cython.

Cython est à la fois un **langage de programmation** très proche de Python ("Cython is Python with C data types"). Il permet deux choses d'obtenir:  
* du **code compilé** à partir de code Python.  
L'idée est de commencer avec du code Python, ajouter des informations sur les types, et obtenir du code compilé plus performant.  
* un **wrapper pour des fonctions C/C++**. 
L'idée est de commencer avec du code compilé (C/C++) et l'exposer à Python avec Cython. 

Les étapes sont les suivantes:
1. écrire un fichier cython (`.pyx`)  
2. exécuter le compiler Cython pour générer du code C  
3. exécuter un compiler C pour générer une librairie compilé
4. exécuter l'interpréteur Python et demander d'importer le module

## Comment intéragir avec Cython?

On va prendre comme exemple une fonction qui renvoie une combinaison linéaire de deux entiers. 

In [ ]:
%%writefile exo_cython/python_add.py
def pyadd(a, b):
    return 3.1415926 * a + 2.718281828 * b

### 1. compiler avec distutils

On commence par d'abord écrire un fichier `.pyx`, en gros du code Python en ajoutant des informations sur le type.

In [ ]:
%%writefile exo_cython/distutils/cython_add.pyx
def add(int a, int b):
    return 3.1415926 * a + 2.718281828 * b

In [ ]:
!cp exo_cython/python_add.py exo_cython/distutils/python_add.py

Il faut ensuite créer un fichier `setup.py` qui est une sorte de Makefile Python et qui fait appel à `distutils` et `cython`:

In [ ]:
%%writefile exo_cython/distutils/setup.py
from distutils.core import setup
from Cython.Build import cythonize

setup(name="cython_add",
      ext_modules = cythonize("cython_add.pyx")
)

On peut maintenant utiliser ce fichier pour compiler le fichier Cython avec la commande:  
`$ python setup.py buil_ext --inplace`

In [ ]:
!cd exo_cython/distutils; python setup.py build_ext --inplace

On obtient alors un fichier `cython_add.so` sous unix (ou `cython_add.pyd` sous Windows) que l'on peut utiliser en l'important dans un interpréteur Python comme n'importe quel autre module Python: `import cython_add`.

Comparons les performances avec la version Python:

In [23]:
%%writefile exo_cython/distutils/compare_add.py
from timeit import timeit
from python_add import pyadd
from cython_add import add

print('Pure Python version: %fs' % timeit('pyadd(1, 2)', 'from python_add import pyadd'))
print('Cython version: %fs' % timeit('add(1, 2)', 'from cython_add import add'))

Overwriting exo_cython/distutils/compare_add.py


In [ ]:
!cd exo_cython/distutils; python compare_add.py

### 2. pyximport

pyximport est un package qui permet d'import un fichier Cython comme si c'était un module Python. Il détecte s'il y a eu des changements dans le fichier Cython, recompile au besoin (ou charge le module du cache). C'est pratique pour les cas simples.

On commence par écrire notre fichier `pyx`:

In [ ]:
%%writefile exo_cython/pyximport/cython_add.pyx
def add(int a, int b):
    return 3.1415926 * a + 2.718281828 * b

Pour utiliser directement le fichier, on utilise:
<code>
import pyximport; pyximport.install()
import cython_add
print(cython_add.add(1,2))
<code>

In [ ]:
!cp exo_cython/python_add.py exo_cython/pyximport/python_add.py

Comparons les performances avec la version Python:

In [ ]:
%%writefile exo_cython/pyximport/compare_add.py
from timeit import timeit
from python_add import pyadd
import pyximport; pyximport.install()
from cython_add import add

print('Pure Python version: %fs' % timeit('pyadd(1, 2)', 'from python_add import pyadd'))
print('Cython version: %fs' % timeit('add(1, 2)', 'from cython_add import add'))

In [ ]:
!cd exo_cython/pyximport; python compare_add.py

### 3. IPython

Il existe une commande magique IPython: 
<code>
%load_ext cythonmagic
%%cython
</code>
Comme avec `pyximport`, la compilation est faite pour nous et c'est une méthode recommandée seulement pour des cas simples.

In [2]:
%load_ext Cython

In [3]:
%%cython
def add(int a, int b):
    return 3.1415926 * a + 2.718281828459045 * b

In [4]:
def pyadd(a, b):
    return 3.1415926 * a + 2.718281828459045 * b

In [8]:
%timeit add(2, 3)

The slowest run took 27.06 times longer than the fastest. This could mean that an intermediate result is being cached 
10000000 loops, best of 3: 79.3 ns per loop


In [9]:
%timeit pyadd(2, 3)

The slowest run took 20.70 times longer than the fastest. This could mean that an intermediate result is being cached 
1000000 loops, best of 3: 196 ns per loop


## Le langage Cython

### 1.1 cdef

`cdef` est la porte d'entrée de C en Python. Il permet de déclarer des variables locales, des fonctions C et des extension types.

In [13]:
%%cython
def add(int a, int b):
    cdef:
        float c
        float d = 2.718281828459045
    c = 3.1415926  
    return c * a + d * b

In [14]:
%timeit add(2, 3)

10000000 loops, best of 3: 79.9 ns per loop


On peut aussir définir une fonction C:
<code>
cdef float def add(int a, int b):
    cdef:
        float c = 3.1415926, d = 2.718281828459045
    return c * a + d * b
<code>

Ou même une extension type (classe bas niveau):  
<code>
cdef class Particle(object): 
    cdef float...
</code>

D'autres declarations cdef:  

cdef      | signification  
--------- | -------------  
cdef int i, j, k | déclare des entiers C   
cdef char *s | déclare une string C  
cdef float x = 0.0 | déclare et initialise un float C  
cdef list names | déclare une liste Python statique  
cdef dict dd = {} | déclare et initialise un dictionnaire Python  

### 1.2 Comment déclarer une fonction en Cython?

Il y a trois manières de déclarer une fonction en Cython:  
* les fonctions `def`: disponibles en Python et Cython. Elles prennent et retournent un objet Python.  
* les fonctions `cdef`: fonction C. On ne peut pas y accéder depuis Python.  
* les fonctions `cpdef`: prennent le meilleur des deux mondes. Deux fonctions sont générées: une fonction C et un wrapper Python autour de cette fonction. 

In [21]:
%%writefile exo_cython/inc1.pyx
def inc(int num, int offset):
    return num + offset

def inc_seq(seq, offset):
    result = []
    for val in seq:
        res = inc(val, offset)
        result.append(res)
    return result

Overwriting exo_cython/inc1.pyx


In [24]:
%%writefile exo_cython/setup1.py
from distutils.core import setup
from Cython.Build import cythonize

setup(name="inc1",
      ext_modules = cythonize("inc1.pyx")
)

Writing exo_cython/setup1.py


In [25]:
!cd exo_cython; python setup1.py build_ext --inplace

Compiling inc1.pyx because it changed.
[1/1] Cythonizing inc1.pyx
running build_ext
building 'inc1' extension
x86_64-linux-gnu-gcc -pthread -fno-strict-aliasing -DNDEBUG -g -fwrapv -O2 -Wall -Wstrict-prototypes -fPIC -I/usr/include/python2.7 -c inc1.c -o build/temp.linux-x86_64-2.7/inc1.o
x86_64-linux-gnu-gcc -pthread -shared -Wl,-O1 -Wl,-Bsymbolic-functions -Wl,-Bsymbolic-functions -Wl,-z,relro -fno-strict-aliasing -DNDEBUG -g -fwrapv -O2 -Wall -Wstrict-prototypes -D_FORTIFY_SOURCE=2 -g -fstack-protector --param=ssp-buffer-size=4 -Wformat -Werror=format-security build/temp.linux-x86_64-2.7/inc1.o -o /home/camille/Documents/Cours/PythonMorpho/exo_cython/inc1.so


In [30]:
%%writefile exo_cython/inc2.pyx
cdef int fast_inc(int num, int offset):
    return num + offset

def fast_inc_seq(seq, offset):
    result = []
    for val in seq:
        res = fast_inc(val, offset)
        result.append(res)
    return result

Overwriting exo_cython/inc2.pyx


In [31]:
%%writefile exo_cython/setup2.py
from distutils.core import setup
from Cython.Build import cythonize

setup(name="inc2",
      ext_modules = cythonize("inc2.pyx")
)

Overwriting exo_cython/setup2.py


In [32]:
!cd exo_cython; python setup2.py build_ext --inplace

Compiling inc2.pyx because it changed.
[1/1] Cythonizing inc2.pyx
running build_ext
building 'inc2' extension
x86_64-linux-gnu-gcc -pthread -fno-strict-aliasing -DNDEBUG -g -fwrapv -O2 -Wall -Wstrict-prototypes -fPIC -I/usr/include/python2.7 -c inc2.c -o build/temp.linux-x86_64-2.7/inc2.o
x86_64-linux-gnu-gcc -pthread -shared -Wl,-O1 -Wl,-Bsymbolic-functions -Wl,-Bsymbolic-functions -Wl,-z,relro -fno-strict-aliasing -DNDEBUG -g -fwrapv -O2 -Wall -Wstrict-prototypes -D_FORTIFY_SOURCE=2 -g -fstack-protector --param=ssp-buffer-size=4 -Wformat -Werror=format-security build/temp.linux-x86_64-2.7/inc2.o -o /home/camille/Documents/Cours/PythonMorpho/exo_cython/inc2.so


In [33]:
!cd exo_cython; python -c 'import inc1; print(inc1.inc(1, 3))'

4


In [35]:
!cd exo_cython; python -c 'import inc1; a = range(4); print(inc1.inc_seq(a, 3))'

[3, 4, 5, 6]


In [37]:
!cd exo_cython; python -c 'import inc2; print(inc2.fast_inc(1, 3))'

Traceback (most recent call last):
  File "<string>", line 1, in <module>
AttributeError: 'module' object has no attribute 'fast_inc'


In [40]:
!cd exo_cython; python -c 'import inc2; a = range(4); print(inc2.fast_inc_seq(a, 3))'

[3, 4, 5, 6]


In [42]:
%%writefile exo_cython/inc3.pyx
cpdef int fast_inc(int num, int offset):
    return num + offset

def inc_seq(seq, offset):
    result = []
    for val in seq:
        res = fast_inc(val, offset)
        result.append(res)
    return result

Overwriting exo_cython/inc3.pyx


In [43]:
%%writefile exo_cython/setup3.py
from distutils.core import setup
from Cython.Build import cythonize

setup(name="inc3",
      ext_modules = cythonize("inc3.pyx")
)

Writing exo_cython/setup3.py


In [44]:
!cd exo_cython; python setup3.py build_ext --inplace

Compiling inc3.pyx because it changed.
[1/1] Cythonizing inc3.pyx
running build_ext
building 'inc3' extension
x86_64-linux-gnu-gcc -pthread -fno-strict-aliasing -DNDEBUG -g -fwrapv -O2 -Wall -Wstrict-prototypes -fPIC -I/usr/include/python2.7 -c inc3.c -o build/temp.linux-x86_64-2.7/inc3.o
x86_64-linux-gnu-gcc -pthread -shared -Wl,-O1 -Wl,-Bsymbolic-functions -Wl,-Bsymbolic-functions -Wl,-z,relro -fno-strict-aliasing -DNDEBUG -g -fwrapv -O2 -Wall -Wstrict-prototypes -D_FORTIFY_SOURCE=2 -g -fstack-protector --param=ssp-buffer-size=4 -Wformat -Werror=format-security build/temp.linux-x86_64-2.7/inc3.o -o /home/camille/Documents/Cours/PythonMorpho/exo_cython/inc3.so


In [45]:
!cd exo_cython; python -c 'import inc3; print(inc3.fast_inc(1, 3))'

4


In [46]:
!cd exo_cython; python -c 'import inc3; a = range(4); print(inc3.inc_seq(a, 3))'

[3, 4, 5, 6]


### 1.3 Exercice de typage

Nous allons nous entraîner à ajouter des informations sur les type.  
La fonction `hamming.py` a deux fonctions simples qui calculent la distance de Hamming entre deux strings: `hamming_sum()` et `hamming_loop()`.

In [49]:
%%writefile exo_cython/hamming.py
def hamming_sum(s0, s1):
    if len(s0) != len(s1):
        raise ValueError()
    return sum(c0 != c1 for (c0, c1) in zip(s0, s1))

def hamming_loop(s0, s1):
    if len(s0) != len(s1):
        raise ValueError()
    count = 0
    for i in range(len(s0)):
        count += (s0[i] != s1[i])
    return count

Writing exo_cython/hamming.py


Ecrire une version Cython `exo_cython/hamming_cython.pyx` en ajoutant des informations sur les types pour agmentant la performance du code. Vous pouvez utiliser `exo_cython/setup_hamming.py` pour compiler votre code et `test_hamming.py` pour comparer les performances.

## 2. Comment interfacer des fonctions C en Python?

On va prendre comme exemple l'interfaçage de la fonction C `strlen`.

In [50]:
%%writefile exo_cython/strlen_module.pyx
# first include the header file you need
cdef extern from "string.h":
    # describe the interface for the functions used
    int strlen(char *c)
    
def get_len(char *message):
    # strlen can now be used from Cython code (but not python)
    return strlen(message)

Writing exo_cython/strlen_module.pyx


In [51]:
%%writefile exo_cython/setup_strlen.py
from distutils.core import setup, Extension
from Cython.Build import cythonize

s = Extension(name="strlen_module",
                sources=["strlen_module.pyx", "strlen.c"])
setup(ext_modules=cythonize([s]))

Writing exo_cython/setup_strlen.py


In [52]:
!cd exo_cython; python setup_strlen.py build_ext --inplace

Compiling strlen_module.pyx because it changed.
[1/1] Cythonizing strlen_module.pyx
running build_ext
building 'strlen_module' extension
x86_64-linux-gnu-gcc -pthread -fno-strict-aliasing -DNDEBUG -g -fwrapv -O2 -Wall -Wstrict-prototypes -fPIC -I/usr/include/python2.7 -c strlen_module.c -o build/temp.linux-x86_64-2.7/strlen_module.o
x86_64-linux-gnu-gcc -pthread -fno-strict-aliasing -DNDEBUG -g -fwrapv -O2 -Wall -Wstrict-prototypes -fPIC -I/usr/include/python2.7 -c strlen.c -o build/temp.linux-x86_64-2.7/strlen.o
x86_64-linux-gnu-gcc: error: strlen.c: No such file or directory
x86_64-linux-gnu-gcc: fatal error: no input files
compilation terminated.
error: command 'x86_64-linux-gnu-gcc' failed with exit status 4


In [58]:
!ls exo_cython

build			     hamming.py   inc3.c	 setup_hamming.py
distutils		     hamming.pyc  inc3.pyx	 setup_strlen.py
hamming_cython.c	     inc1.c	  inc3.so	 strlen_module.c
hamming_cython.pyx	     inc1.pyx	  python_add.py  strlen_module.pyx
hamming_cython.so	     inc1.so	  pyximport	 test_hamming.py
hamming_cython_solution.c    inc2.c	  setup1.py
hamming_cython_solution.pyx  inc2.pyx	  setup2.py
hamming_cython_solution.so   inc2.so	  setup3.py


In [57]:
!cd exo_cython; python -c 'import strlen_module; strlen_module.strlen'

Traceback (most recent call last):
  File "<string>", line 1, in <module>
ImportError: No module named strlen_module
